In [29]:
import requests
import pandas as pd
import time

In [30]:
key = '80d9de17fa194ad2804602599b9601be'
endpoint = "https://api.cognitive.microsoft.com/bing/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": key}
params = {"q": '', "textDecorations": False, "textFormat": "HTML", 'count': 3,'responseFilter':'Webpages',}

In [57]:
df = pd.read_csv('district_sample_web_catalog - schools_cleaned.csv', index_col = 0).astype(
'object')
df2 = pd.read_csv('school_sample_covid_cleaned.csv', index_col = 0).astype('object')
df2 = df2.drop(columns = ['district','state','school','school_sample'])
df = df.merge(df2,how = 'inner',on = ['cleaned'])
df['zip']=df['zip'].apply(lambda x: str(x).zfill(5))

In [58]:
#Currently created manually, might be better once we can run a lot of testing to just filter out domains that come up frequently
url_filters = ['facebook.com','elementaryschools.org','usnews.com','greatschools.org',
              'niche.com','schooldigger.com','wikipedia.org','publicschoolreview.com','areavibes.com','realtor.com','high-school.com']
database_sites = ['greatschools.org','niche.com','schooldigger.com','elementaryschools.org','publicschoolreview.com']

In [59]:
def name_city(row):
    search_term = row['cleaned']
    if row['city'] not in search_term:
        search_term += ' '+ row['city']
    return search_term

def name(row):
    search_term = row['cleaned']
    return search_term
def name_city_zip(row):
    search_term = row['cleaned']
    if row['city'] not in search_term:
        search_term += ' '+ row['city']
    search_term += ' ' + row['zip']
    return search_term

In [60]:
def whitelisted(url_string):
    return any(website in url_string for website in url_filters)
def is_database(url_string):
    return any(website in url_string for website in databse_sites.values())

In [61]:
def get_url(row,searchTermFunc):
    params['q'] = searchTermFunc(row)
    response = requests.get(endpoint, headers=headers, params=params)
    search_results = response.json()
    urls = [search_results['webPages']['value'][i]['url'] for i in range(3)]
    #To prevent calling the API too many times per second
    #time.sleep(.5)
    if not any(check in urls[0] for check in url_filters):
        return [urls[0],None]
    for url in urls:
        if any(check in url for check in database_sites):
            return [None,url]
    return [None,None]

In [62]:
%%timeit -r1 -n1
#Be careful running this as it performs as many API calls as the test_schools_df and we have limited amounts of calls
results = df.apply(get_url,args = [name_city],axis=1)
df['bing_url'],df['alt_url'] = map(list, zip(*results)) 
results = df.apply(get_url,args = [name_city_zip],axis=1)
df['zip_url'],df['zip_alt'] = map(list, zip(*results)) 

1min 6s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [63]:
df.head()

,district,state,school,level,school_link,different from district?,learning platform?,learning platform_name,date collected,pdf,...,collected by,cleaned,city,county,address,zip,bing_url,alt_url,zip_url,zip_alt
0,Adelanto Elementary,CA,Eagle Ranch Elementary,Elementary,https://www.aesd.net/ers/home,0.0,Nan,Nan,03/30/2020,0,...,Cathy,Eagle Ranch Elementary School,Adelanto,San Bernardino County,12545 Eagle Ranch Pkwy.,92301,http://www.ers.aesd.net/,None,http://www.ers.aesd.net/,None
1,Adelanto Elementary,CA,Adelanto Elementary,Elementary,https://www.aesd.net/aes/home,0.0,Nan,Nan,03/30/2020,0,...,Cathy,Adelanto Elementary School,Adelanto,San Bernardino County,17931 Jonathan St.,92301,http://www.aesd.net/,None,http://www.aes.aesd.net/,None
2,Adelanto Elementary,CA,Melva Davis Academy Of Excellence,Middle,https://www.aesd.net/13/Home,0.0,Nan,Nan,03/30/2020,0,...,Cathy,Melva Davis Academy Of Excellence,Victorville,San Bernardino County,15831 Diamond Rd.,92394,http://www.aesd.net/mdae/home,None,http://www.aesd.net/mdae/home,None
3,Adelanto Elementary,CA,Mesa Linda Middle,Middle,https://www.aesd.net/mlms/home,0.0,Nan,Nan,03/30/2020,0,...,Cathy,Mesa Linda Middle School,Victorville,San Bernardino County,13001 Mesa Linda Rd.,92392,http://www.mlms.aesd.net/,None,http://www.mlms.aesd.net/,None
4,Baltimore City Public Schools,MD,Hamilton Elementary/Middle,Elementary,https://www.hamiltonelementarymiddle.org/,Nan,Nan,Nan,03/30/2020,Nan,...,Cathy,Hamilton Elementary/Middle School,Baltimore,Baltimore City,6101 Old Harford Rd,21214,https://www.baltimorecityschools.org/schools/236,None,https://www.hamiltonelementarymiddle.org/,None


In [26]:
count = 0
for i, row in df.iterrows():
    bing = row['bing_url']
    zip_url = row['zip_search']
    if correct == 'NA':
        correct = None
    if bing == 'None':
        bing = None
    if correct == bing:
        count +=1

In [27]:
for i, row in df.loc[:25].iterrows():
    print(row['school_link'],row['bing_url'])

https://www.aesd.net/ers/home http://www.ers.aesd.net/
https://www.aesd.net/aes/home http://www.aesd.net/
https://www.aesd.net/13/Home http://www.aesd.net/mdae/home
https://www.aesd.net/mlms/home http://www.mlms.aesd.net/
https://www.hamiltonelementarymiddle.org/ https://www.baltimorecityschools.org/schools/236
https://libertyelementary64.weebly.com/home-resources.html http://www.libertylearns.com/
https://www.edmondson-westside.org/apps/events2/event.jsp?eREC_ID=2891782&d=2020-03-30&id=0 https://www.edmondson-westside.org/
http://www.fdhsbaltimore.org/ http://www.fdhsbaltimore.org/
nan https://www.baltimorecityschools.org/schools/130
https://basdwpweb.beth.k12.pa.us/fountainhill/ https://basdwpweb.beth.k12.pa.us/fountainhill/
https://basdwpweb.beth.k12.pa.us/thomasjefferson/ https://basdwpweb.beth.k12.pa.us/thomasjefferson/
https://basdwpweb.beth.k12.pa.us/freedom/ https://basdwpweb.beth.k12.pa.us/freedom/
https://basdwpweb.beth.k12.pa.us/liberty/ https://basdwpweb.beth.k12.pa.us/libe

In [28]:
#df.to_csv('test.csv')

In [64]:
correct = pd.read_csv('test - test.csv')
df = df.merge(correct, on = 'cleaned')

In [76]:
pop = df[df['bing_url'] != df['zip_url']]

In [77]:
pop[pop['Correct']==1]

,district,state,school,level,school_link,different from district?,learning platform?,learning platform_name,date collected,pdf,...,county,address,zip,bing_url,alt_url,zip_url,zip_alt,Correct,District Site,Couldve been correct
5,Baltimore City Public Schools,MD,Liberty Elementary,Elementary,https://libertyelementary64.weebly.com/home-re...,1.0,Nan,Nan,03/30/2020,0,...,Baltimore City,3901 Maine Ave,21207,http://www.libertylearns.com/,None,None,https://www.greatschools.org/maryland/baltimor...,1.0,NaN,NaN
33,Frisco Isd,TX,Christie El,Elementary,http://schools.friscoisd.org/campus/elementary...,1.0,1.0,"Google Docs, Youtube",03/31/2020,0,...,Collin County,10300 HUNTINGTON RD,75035,None,https://www.greatschools.org/texas/frisco/8676...,None,https://www.greatschools.org/texas/frisco/8676...,1.0,NaN,NaN
48,Herricks Union Free School District,NY,Center Street School,Elementary,https://www.herricks.org/Domain/553,Nan,0.0,Nan,03/31/2020,0,...,Nassau County,240 CENTER ST,11596,https://www.herricks.org/center,None,https://www.redfin.com/NY/Williston-Park/106-C...,None,1.0,NaN,NaN
58,Laurel County,KY,Bush Elementary School,Elementary,https://www.laurel.k12.ky.us/5/Home,1.0,1.0,Iready,03/31/2020,0,...,Laurel County,1832 E Laurel Rd,40741,http://www.laurel.k12.ky.us/school_home.aspx?s...,None,https://www.laurel.kyschools.us/5/Home,None,1.0,NaN,NaN
60,Laurel County,KY,North Laurel High School,High,https://www.laurel.k12.ky.us/1/Home,0.0,Nan,Nan,03/31/2020,0,...,Laurel County,1300 East Hal Rogers Parkway,40741,http://www.laurel.k12.ky.us/school_home.aspx?s...,None,http://www.laurel.k12.ky.us/1/Home,None,1.0,NaN,NaN
68,Montgomery County Public Schools,MD,Chevy Chase Elementary,Elementary,https://www2.montgomeryschoolsmd.org/schools/c...,1.0,Nan,Nan,03/31/2020,0,...,Montgomery County,4015 Rosemary St,20815,https://www2.montgomeryschoolsmd.org/schools/c...,None,https://www2.montgomeryschoolsmd.org/schools/n...,None,1.0,NaN,NaN
88,Pcm Comm School District,IA,Prairie City Elementary School,Elementary,https://pce.pcmschools.org,0.0,1.0,Mixed Curriculum,03/31/2020,0,...,Jasper County,309 East Plainsmen Road,50228,https://pce.pcmschools.org/,None,http://pce.pcmschools.org/,None,1.0,NaN,NaN
94,Prince George'S County Public Schools,MD,Central High,High,https://www.pgcps.org/coronavirus/,0.0,Nan,Nan,03/30/2020,0,...,Prince George'S County,200 Cabin Branch Rd,20743,http://www1.pgcps.org/central/,None,None,https://www.greatschools.org/maryland/capitol-...,1.0,NaN,NaN
106,Westville,OK,Westville Jhs,Middle,https://www.gowestville.org/o/wjh,Nan,Nan,Nan,03/30/2020,Nan,...,Adair County,500 W Chincapin,74965,http://www.gowestville.org/o/wjh,None,http://www.westville.k12.ok.us/,None,1.0,NaN,NaN
110,Yuba City Unified,CA,Yuba City High,High,http://ychs.ycusd.org/documents/Message%20from...,1.0,Nan,Nan,03/30/2020,1,...,Sutter County,850 B St.,95991,http://ychs.ycusd.org/,None,http://www.ycusd.org/,None,1.0,NaN,NaN
